In [72]:
import os
import langchain_chroma
import openai
import chromadb
import fitz  # PDF에서 텍스트 추출
from dotenv import load_dotenv
from fastapi import FastAPI, UploadFile, File
from sqlalchemy import create_engine, Column, Integer, String, Text, TIMESTAMP
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime
from sqlalchemy import create_engine, text
import pandas as pd
import psycopg2
from openai import OpenAI
# .env 파일 로드
load_dotenv()

True

In [3]:
load_dotenv()

True

In [14]:
POST_DB_HOST = os.getenv("POST_DB_HOST")
POST_DB_NAME = os.getenv("POST_DB_NAME")
POST_DB_USER = os.getenv("POST_DB_USER")
POST_DB_PASSWD = os.getenv("POST_DB_PASSWD")
POST_DB_PORT = os.getenv("POST_DB_PORT")

# PostgreSQL 연결

In [4]:
POST_DB_HOST

'43.202.186.183'

In [5]:
# PostgreSQL 연결 엔진 생성
post_engine = create_engine(f'postgresql://{POST_DB_USER}:{POST_DB_PASSWD}@{POST_DB_HOST}:{POST_DB_PORT}/{POST_DB_NAME}')

In [15]:
db = psycopg2.connect(host=POST_DB_HOST, dbname=POST_DB_NAME,user=POST_DB_USER,password=POST_DB_PASSWD,port=POST_DB_PORT)
cursor = db.cursor()

In [29]:
db.autocommit = False

# pdf, pdf to text 데이터 

In [31]:
path = "./무용수_이력서.pdf"
doc = fitz.open(path)
for page in doc:
    resume_text = page.get_text()
    print(resume_text)

무용수 이력서
무용수 신입 이력서 예시
📌 기본 정보
이름: 김예린
연락처: 010-5678-1234
이메일: yerin.kim@example.com
인스타그램/포트폴리오: @yerin_dance
키: 168cm / 몸무게: 52kg
📌 자기소개 (About Me)
무대 위에서 감정을 온전히 표현하는 무용수 김예린입니다. 현대무용과 발레를 중심으로 다
양한 퍼포먼스 경험을 쌓아왔으며, 작품 해석 능력과 유연한 움직임이 강점입니다. 끊임없는 
연습과 창의적인 안무 해석으로 무대에서 감동을 선사하는 아티스트가 되고 싶습니다.
📌 학력
○ 한국예술종합학교 무용원 현대무용과
재학기간: 2019.03 ~ 2025.02 (졸업예정)
전공: 현대무용
주요 수업: 테크닉 트레이닝, 즉흥무, 안무 창작, 무용 분석
📌 주요 경력 및 활동
1. 2024 서울 국제댄스페스티벌 참가
안무가 OOO의 작품 "Beyond the Silence" 출연
섬세한 감정 연기와 유려한 동작 표현으로 호평받음
2. 2023 무용 창작 공연 "몸의 언어" 참여
창작안무 및 솔로 공연 진행
감각적인 움직임과 공간 활용
무용수 이력서
1



In [64]:
path = "./resume_sample.pdf"
doc = fitz.open(path, filetype="pdf")
text = "\n".join([page.get_text("text") for page in doc])

'이력서\nQA 엔지니어 신입 이력서\n📌 기본 정보\n이름: 김지원\n연락처: 010-1234-5678\n이메일: jiwoon.kim@example.com\nGitHub: github.com/jiwoonkim\nLinkedIn: linkedin.com/in/jiwoonkim\n📌 자기소개 (About Me)\n소프트웨어 품질 보증(QA)에 대한 깊은 관심과 꼼꼼한 분석력을 바탕으로, 사용자에게 신뢰\n할 수 있는 서비스를 제공하는 것을 목표로 하는 QA 엔지니어 신입 김지원입니다. 테스트 자\n동화 및 결함 분석을 통해 제품의 완성도를 높이는 데 기여하고 싶습니다.\n📌 학력\n○ 서울대학교 컴퓨터공학과\n재학기간: 2019.03 ~ 2025.02 (졸업예정)\n주요과목: 소프트웨어 공학, 데이터베이스, 운영체제, 알고리즘, 테스트 자동화\n📌 기술 스택\n✅ 테스트 자동화: Selenium, JUnit, PyTest\n✅ 프로그래밍 언어: Python, Java, JavaScript\n✅ CI/CD: Jenkins, GitHub Actions\n✅ 테스트 도구: Postman(REST API 테스트), JMeter(부하 테스트)\n✅ 버전 관리: Git, GitHub, GitLab\n✅ 이슈 트래킹: JIRA, Trello\n📌 프로젝트 경험\n이력서\n1\n\n1. 웹 서비스 자동화 테스트 프로젝트\n기간: 2024.03 ~ 2024.06\n내용: Selenium과 PyTest를 활용하여 웹 애플리케이션 자동화 테스트 진행\n성과:\nUI 테스트 자동화로 테스트 소요 시간 40% 단축\n30개 이상의 주요 기능 테스트 케이스 작성 및 실행\n2. 모바일 앱 기능 테스트 (Android, iOS)\n기간: 2023.12 ~ 2024.02\n내용: Appium을 이용한 모바일 애플리케이션 UI 테스트\n성과:\n15개 주요 기능 테스트 시나리오 작성 및 자동화\n크리티컬 버그 5건 발견 및 개발팀과 협업하여 해결\n📌 인턴 및 실무 경험\n○ [ABC 테크]

In [47]:
user_id = 'interview'

In [ ]:
sql = """
INSERT INTO resume_popol_history (user_id, resume_text, cover_letter_text, popol_text)
VALUES (%s, %s, %s, %s);
"""

In [60]:
cursor.execute(sql, (user_id, resume_text))

In [ ]:
sql = """
DELETE INTO resume_popol_history (user_id, resume_text)
VALUES (%s, %s);
"""

In [61]:
db.commit()

In [ ]:
cursor.close()
db.close()

In [50]:
db.rollback()

In [ ]:
def insert_data():
    sql = """
    INSERT INTO resume_popol_history (user_id, resume_text, cover_letter_text, popol_text)
    VALUES (%s, %s, %s, %s);
    """
    cursor.execute(sql, (user_id, resume_text, cover_letter_text, popol_text))
    db.commit()
    cursor.close()
    db.close()


In [ ]:
# 마지막 삽입 데이터 삭제 / 왜쓰는지 알아보자
def del_last_data():
    sql = """
    delete from resume_popol_history 
        where user_id = %s
        and insert_date = ( select insert_date
                                from resume_popol_history rph
                                order by insert_date desc
                                limit 1
                                    );
    """
    cursor.execute(sql,(user_id))
    db.commit()
    cursor.close()
    db.close()

# tokenizing before embedding 

In [18]:
sql = """
        select rec_idx, company_nm, jd_text from saramin_recruit_detail
"""

cursor.execute(sql)


In [19]:
jd_text_list = cursor.fetchall()

In [45]:
import openai
import pandas as pd

In [46]:
# 텍스트 임베딩 함수
def get_embedding(text, model="text-embedding-3-large"):
    response = openai.embeddings.create(
        text = resume_text.replace("\n", "").strip(),
        model=model,
        input=text
    )
    return response.data[0].embedding

In [56]:
pd.DataFrame(jd_text_list)[2].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
13848    False
13849    False
13850    False
13851    False
13852    False
Name: 2, Length: 13853, dtype: bool

# 자 여기부터 tokenizing

# 여긴 리스트 임베딩 

In [ ]:
# from openai import OpenAI

In [74]:
client = OpenAI()

In [67]:
test = jd_text_list[1]

In [77]:
embed_test = test[2]

In [73]:
def get_embedding_list(texts, model="text-embedding-3-small"):
   return client.embeddings.create(input = texts, model=model).data

In [79]:
texts = embed_test

embedding_list = get_embedding_list(texts)
print(embedding_list)

[Embedding(embedding=[-0.03588516265153885, 0.02009727992117405, 0.013563677668571472, 0.014735356904566288, 0.059854939579963684, -0.011359332129359245, -0.023989638313651085, 0.04170384258031845, -0.04333227500319481, -0.008296085521578789, 0.03272758424282074, -0.02122924104332924, -0.022520074620842934, 0.059656351804733276, 0.01408994011580944, -0.0031699882820248604, -0.046668585389852524, -0.02355273999273777, -0.001227532746270299, 0.025479061529040337, 0.023036407306790352, -0.03759303316473961, -0.004205137491226196, -0.03630220144987106, 0.030801264569163322, -0.022301625460386276, 0.0171978697180748, 0.023155560716986656, -0.0040909480303525925, -0.005327169317752123, 0.03610360994935036, -0.0200873501598835, -0.01377219706773758, -0.02669045887887478, 0.008152107708156109, -0.004346632398664951, 0.0035696502309292555, 0.032628290355205536, -7.458750769728795e-05, -0.042974814772605896, -0.00539171090349555, -0.0018208194524049759, -0.0186376441270113, -0.0316154845058918, 

In [ ]:
for i in range(len(texts)):
    print(embedding_list[i].embedding)

In [52]:
print(type(jd_text_list))
jd_texts = [text for text in jd["jd_text"].tolist() if isinstance(text, str) and text.strip()]


<class 'list'>


# embedding text

In [ ]:
model = 'text-embedding-3-large'

In [ ]:
# 텍스트 임베딩 함수
def get_embedding(text, model="text-embedding-3-large"):
    response = openai.embeddings.create(
        text = resume_text.replace("\n", "").strip(),
        model=model,
        input=text
    )
    return response.data[0].embedding

In [ ]:
# 예시
embedding = get_embedding(text)